Thinking1 什么是近似最近邻查找，常用的方法有哪些
答：近似最近邻查找为ANN（approximate nearest neighbor）,相对于KNN（K-Nearest Neighbor）精确查找top-n个近邻的方法来说，是一种近似的查找方法，为了在海量的数据集里用更少的计算成本来更快的获得目标item的近邻的方法，相比较KNN，ANN并没有精确计算所有数据集两两之间的距离，是利用局部敏感哈希（LSH）的方法计算得出近似最近邻的检索方式。LSH常用方式包括MinHash和SimHash,MinHash是通过对原有集合进行多次随机变换，并计算每次变换之后集合的MinHash值，然后计算这个minhash数据集两两向量之间的jaccard相似度，并用这个jaccard相似度近似代替原有数据集两两之间的jaccard相似度进行后续的相似度计算。而SimHash是通过将所有item的feature分为多个数量相等的band,然后计算每个item在每个band上的哈希映射，将所有的item最终都映射到这每个band里的不同桶里，计算相似度时，对每个band都从各自的桶里通过计算汉明距离捞出近似的候选集，在实际使用中可能会再对候选集做一次精确查找，最终返回相似度大的数据集。

Thinking2 为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度
答：两个集合的Jaccard相似度为交集/并集，假设两个item的所有feature都是独热编码，
假设某个feature其两个item的值都为1，则称该行为A类行，如果某一行，两个item中只有一个为1，另一个为0，则称其为B类行，若某行item都为0，则称其为C行，则两个item之间的Jaccard相似度为A/(A+B)。minhash值的计算方式是特征矩阵按行进行随机排列后，第一个列值为1的行的行号，所以C类行如何变换都对结果没有影响,删掉C类行后，两个item之间minhash相等的概率也等于A/(A+B)，所以说两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度。

Thinking3 SimHash在计算文档相似度的作用是怎样的？
答：SimHash在计算文档相似度的作用主要是降维和计算生成每个文档的指纹，将很高维的文本特征计算固定到相同的比较小的维度上，并且保持每个文档生成的指纹能够尽可能保持原有文档的特征，这样在后续计算相似度上就能大大减少时间成本以及内存成本。

Thinking4 为什么YouTube采用期望观看时间作为评估指标
答：YouTube为视频网站，用户在某个视频上逗留的时间越长，越能够将视频中的所携带的附带效益最大化，例如视频中可能存在中插广告，如果用户在某个视频逗留时间不长，甚至只是点击后就离开，用户很可能就看不到这些中插广告，那么对于广告商来说，广告无法为他们带来流量就认为说中插广告不值得投入，那么对于YouTube来说，就会损失很多这方面的效益。

In [20]:
import jieba.posseg as pseg
import re
import pandas as pd

In [2]:
#weibo=pd.read_csv("G:\\python_lesson\\L6-code\\L6\\weibos.txt",sep='\t')
#print(type(weibo))
#open for reading，只读模式
f = open('G:\\python_lesson\\L6-code\\L6\\weibos.txt', 'r', encoding='UTF-8')
#print(type(f))
weibo=f.read()
#print(weibo)
weibo_sentences=re.split('[#。！？……]',weibo.replace('\n',""))
#print(weibo_sentences)
#print(len(weibo_sentences))

In [3]:
#去掉空值
weibo_sentences = [i for i in weibo_sentences if i != '']
print(weibo_sentences)
print(len(weibo_sentences))

['斯科拉里愿意执教国足', '上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用', ' \u200b国足输给叙利亚之后，里皮辞职', '谁将成为新主帅，成为广大球迷关注的焦点', '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里', ' \u200b据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决', '据了解', '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战', '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利', '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话', '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商', '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢', '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足', '老头有点意思，凡是里皮干过的地方，他就想试试', '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛', '可以试试', '斯科拉里的水平，还不如里皮', '斯科拉里，看好的不是国足，而是年薪', ' \u200b非常应该辞职', '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有', '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了', '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光', '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队', '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去', '”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛', '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州', '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事

In [4]:
#加载停用词
stop=[]
for word in open('G:\\python_lesson\\L6-code\\L6\\chineseStopWords.txt','r'):
    stop.append(word.strip())
print(stop)

['?', '、', '。', '“', '”', '《', '》', '!', '！', '，', ',', '：', ':', '；', '？', '-', '(', ')', '（', '）', '·', '--', '……', '/', '.', '|', '——', '‘', '’', '□', '【', '】', 'A', 'B', 'C', 'D', '啊', '阿', '哎', '哎呀', '哎哟', '唉', '俺', '俺们', '按', '按照', '吧', '吧哒', '把', '罢了', '被', '本', '本着', '比', '比方', '比如', '鄙人', '彼', '彼此', '边', '别', '别的', '别说', '并', '并且', '不比', '不成', '不单', '不但', '不独', '不管', '不光', '不过', '不仅', '不拘', '不论', '不怕', '不然', '不如', '不特', '不惟', '不问', '不只', '朝', '朝着', '趁', '趁着', '乘', '冲', '除', '除此之外', '除非', '除了', '此', '此间', '此外', '从', '从而', '出', '打', '待', '但', '但是', '当', '当着', '到', '得', '的', '的话', '等', '等等', '地', '第', '叮咚', '对', '对于', '多', '多少', '而', '而况', '而且', '而是', '而外', '而言', '而已', '尔后', '反过来', '反过来说', '反之', '非但', '非徒', '否则', '嘎', '嘎登', '刚', '刚刚', '该', '赶', '个', '各', '各个', '各位', '各种', '各自', '给', '根据', '跟', '故', '故此', '固然', '关于', '管', '归', '果然', '果真', '过', '哈', '哈哈', '呵', '和', '何', '何处', '何况', '何时', '嘿', '哼', '哼唷', '呼哧', '乎', '哗', '还是', '还有', '换句话说', '换言之', '或', '或是', '或者', '极了', '及', '及其', '及

In [5]:
#jieba分词并去掉停用词
def get_item_str(item_text):
    item_str = ""
    item=(pseg.cut(item_text))
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

In [6]:
#分词
documents=[]
for sentence in weibo_sentences:
    documents.append(get_item_str(sentence))
print(documents)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lyr\AppData\Local\Temp\jieba.cache
Loading model cost 1.943 seconds.
Prefix dict has been built successfully.


['斯科拉里 执教 国 足 ', '一届 皮 从头 芾 尾 很大 机会 入 世界杯 一届 没 几个 能用 归化 没用 ', '  \u200b 国 足 输给 叙利亚 皮 辞职 ', '新 主帅 球迷 关注 焦点 ', '舆论 倾向 三个 山东鲁能 主帅 李霄鹏 武汉 卓尔 主帅 李铁 广州 恒大 主帅 斯科拉里 ', '  \u200b 中国足协 态度 皮 请辞 意 决 ', '', '比赛 当晚 太太 西蒙内塔 女士 儿子 小里皮 现场 台上 观战 ', '辞职 里皮 改变 原有 计划 — — 赛后 第二天 迪拜 飞 回 意大利 ', '意味着 本来 没 打算 球队 管理层 中国足协 高层 赛后 时间 辞职 对话 ', '辞职 善后工作 包括 合同 沟通 工作 要待 日后 进一步 协商 ', '回顾 国 足 历届 外籍 教练 — — 皮 佩兰 卡马乔 杜伊 科维奇 阿里 汉 米卢 ', '一个 一个 有名 一个 一个 水 国足 踢 不好 足协 足协 解散 重组 天王老子 请来 不行 斯科拉里 想 执教 中国 国 足 ', '老头 凡是 皮 干 地方 想 试试 ', '老头 世界杯 冠军 教头 折 中国 没 丢人 毕竟 皮 折 ', '试试 ', '斯科拉里 水平 皮 ', '斯科拉里 看好 国 足 年薪 ', '  \u200b 辞职 ', '中国 足球 名帅 外籍 教练 一点儿 毛用 ', '施拉普纳 二十余年 间 中国 足球 竟然 大踏步 倒退 一点儿 杀 不住 车 奶奶 刹车 系统 坏 ', '穿着 几百块 钱 球衣 几千块 钱 球鞋 几万块 钱 包 几十万 包机 几百万 千万 年薪 赛后 叙利亚 主教练 更衣室 队员 一个 耳光 ', '主教练 赛前 老子 再三 交代 一场 无论如何 赢 中国队 ', '中国 援助 粮食 美金 援助 国家 狗日 些 吃 土 ', '球员 委屈 七十多 分钟 晓得 龟儿子 家 球门 踢 ', '皮 辞职 返回 意大利 助教 马达 洛尼 随队 返回 广州 ', '马达 洛尼 接受 采访 还原 更衣室 情况 更衣室 球员 试图 说服 皮 收回 队长 郑智 尝试 阻止 足协 代表 希望 考虑一下 建议 重新考虑 无济于事 ', '中国足协 接受 皮 辞职 请求 深刻反思 报道 马达 洛尼 皮 辞职 事先 告知 发生 皮 头脑 一热

In [13]:
from datasketch import MinHash,MinHashLSHForest
def get_minhash(item_str):
    temp=MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [15]:

minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    temp=get_minhash(documents[i])
    #print(temp)
    minhash_list.append(temp)
    forest.add(i,temp)
forest.index()

In [16]:
query = '斯科拉里愿意执教国足'
query_str = get_item_str(query)
print(query_str)

斯科拉里 执教 国 足 


In [19]:
minhash_query = get_minhash(query_str)
result = forest.query(minhash_query, 3)
print(result)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

[0, 17, 32]
0 1.0 斯科拉里执教国足
17 0.5625 斯科拉里看好国足年薪
32 0.3984375 支持斯科拉里
Top 3 邻居 [0, 17, 32]
